## AgentOps Integration Guide - Step by Step
### Overview
This notebook demonstrates how to integrate AgentOps with OpenAI's API to track and monitor AI agent interactions. AgentOps provides observability for AI agents, allowing you to monitor performance, costs, and agent behaviors.

## What is AgentOps?
AgentOps is a comprehensive monitoring and observability platform specifically designed for AI agents and multi-agent systems. It acts as a "mission control" for your AI applications, providing real-time insights into how your agents are performing, what they're doing, and how much they're costing you.

## Why Use AgentOps?
Without AgentOps, when you deploy AI agents, you're essentially flying blind:

* ❌ No visibility into agent decision-making processes
* ❌ No cost tracking across multiple API calls
* ❌ Difficult to debug when agents fail or behave unexpectedly
* ❌ No performance metrics to optimize your agents
* ❌ Hard to track which agent did what in multi-agent scenarios

With AgentOps, you get complete visibility:

* ✅ Real-time Monitoring: See exactly what your agents are doing as they do it
* ✅ Cost Tracking: Monitor token usage and costs across all your AI interactions
* ✅ Performance Analytics: Track response times, success rates, and error patterns
* ✅ Agent Attribution: Know which specific agent performed each action
* ✅ Session Management: Group related agent activities for easier analysis
* ✅ Debugging Tools: Detailed logs and traces for troubleshooting

## Step 1: Installation and Dependencies

In [15]:
!pip install agentops openai python-dotenv

### Purpose: Install the required packages:

agentops: The main monitoring and observability library
openai: OpenAI's official Python client
python-dotenv: For loading environment variables from .env files

## Step 2: Import Required Libraries

In [ ]:
import agentops
from agentops.sdk.decorators import agent, operation
from openai import OpenAI
import os
from dotenv import load_dotenv
import logging
from IPython.display import display, Markdown

## Step 3: Environment Setup and Configuration

In [3]:
load_dotenv()
OPENAI_API_KEY = "Your_Opean_AI_API_Key"
AGENTOPS_API_KEY = "Your_AGENTOPS_API"
logging.basicConfig(
    level=logging.DEBUG
)

## Step 4: Initialize Services

In [4]:
openai = OpenAI(api_key=OPENAI_API_KEY)
agentops.init(AGENTOPS_API_KEY, tags=["openai-gpt-notebook"])
print("✅ Setup complete!")

🖇 AgentOps: [OPENAI INSTRUMENTOR] Error setting up OpenAI streaming wrappers: No module named 'openai.resources.beta.chat'
🖇 AgentOps: AgentOps: Successfully instrumented 'OpenaiInstrumentor' for package 'agentops.instrumentation.providers.openai'.


✅ Setup complete!


## Step 5: Basic OpenAI API Call with Monitoring

In [5]:
message = [{"role": "user", "content": "Write a 12 word poem about secret agents."}]
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", messages=message, temperature=0.5, stream=False
)
print(response.choices[0].message.content)

Shadows in the night, whispers in the dark, secret agents embark.


## Step 6: End Session

In [6]:
agentops.end_session("Success")

🖇 AgentOps: end_session() is deprecated and will be removed in v4 in the future. Use agentops.end_trace() instead.
🖇 AgentOps: end_session called but no active trace context found.


## Step 7: Multi-Agent Architecture Setup
### Initialize New Session

In [7]:
agentops.init(AGENTOPS_API_KEY, tags=["multi-agent-notebook"])
openai_client = OpenAI(api_key=OPENAI_API_KEY)

## Create QA Agent Class

In [8]:
@agent(name="qa")
class QaAgent:
    def __init__(self):
        pass

    @operation
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a qa engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content


## Create Engineer Agent Class

In [9]:
@agent(name="engineer")
class EngineerAgent:
    def __init__(self):
        pass

    @operation
    def completion(self, prompt: str):
        res = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a software engineer and only output python code, no markdown tags.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.5,
        )
        return res.choices[0].message.content

## Step 8: Instantiate Agents

In [10]:
qa = QaAgent()
engineer = EngineerAgent()

## Step 9: Generate Code with Engineer Agent

In [11]:
generated_func = engineer.completion("python function to test prime number")

## Step 10: Display Generated Code

In [12]:
display(Markdown("```python\n" + generated_func + "\n```"))

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True
```

## Step 11: Generate Tests with QA Agent

In [13]:
generated_test = qa.completion(
    "Write a python unit test that test the following function: \n " + generated_func
)

## Step 12: Display Generated Tests

In [14]:
display(Markdown("```python\n" + generated_test + "\n```"))

```python
import unittest

def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num ** 0.5) + 1):
        if num % i == 0:
            return False
    return True

class TestIsPrime(unittest.TestCase):

    def test_is_prime(self):
        self.assertFalse(is_prime(1))
        self.assertTrue(is_prime(2))
        self.assertTrue(is_prime(3))
        self.assertTrue(is_prime(5))
        self.assertFalse(is_prime(4))
        self.assertFalse(is_prime(6))
        self.assertTrue(is_prime(7))

if __name__ == '__main__':
    unittest.main()
```

# Traces

Now Go to AgentOps website from where you have take the api and go to Traces Section you can see all the traces of the Agent

<img src="images/Screenshot 2025-07-24 100720.png">

<img src="images/Screenshot 2025-07-24 100820.png">